In [1]:
from selenium import webdriver
import pandas as pd
import os
import datetime
from multiprocessing.pool import ThreadPool
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
chromeOptions = Options()

In [38]:
def get_stock_url(url):
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
        )
    driver.get(url)
    element = driver.find_element(By.XPATH, "//div[@class='holdings101Cta cur-po']")
    element.click()
    table = driver.find_element(By.XPATH, "//table[@class='tb10Table holdings101Table']")
    trs = table.find_elements(By.TAG_NAME, 'tr')
    stock_names = []
    stock_url = []
    for i in range(1, len(trs)):
        try:
            name = trs[i].find_element(By.TAG_NAME, "td").text
            link = trs[i].find_element(By.TAG_NAME, "a").get_attribute("href")
            stock_names.append(name)
            stock_url.append(link)
        except:
            continue
    driver.close()
    df = pd.DataFrame({"Name":stock_names,
                       "Link": stock_url
                       })
    return df



In [39]:
df = get_stock_url(url)

In [40]:
df.head()

,Name,Link
0,Reliance Industries Ltd.,https://groww.in/stocks/reliance-industries-ltd
1,ITC Ltd.,https://groww.in/stocks/itc-ltd
2,HDFC Bank Ltd.,https://groww.in/stocks/hdfc-bank-ltd
3,Jindal Stainless Ltd.,https://groww.in/stocks/jindal-stainless-ltd
4,RBL Bank Ltd.,https://groww.in/stocks/rbl-bank-ltd


In [49]:
def get_stock_urls():
    today = str(datetime.date.today())
    root_dir = "data/grow"
    path = os.path.join(root_dir, today, "index", "index.csv")
    index = pd.read_csv(path)
    # dfs = map(get_stock_url, index["link"])
    pool = ThreadPool(os.cpu_count())
    results = pool.map(get_stock_url,index["link"])
    results = [result.get() for result in results]
    df = pd.concat(results)
    return df

In [51]:
df = get_stock_urls()

FileNotFoundError: [Errno 2] No such file or directory: 'data/grow/2023-04-26/index/index.csv'